clone and cd into repo

In [7]:
# first download the gpt-2 code
!git clone https://github.com/nshepperd/gpt-2.git

fatal: destination path 'gpt-2' already exists and is not an empty directory.


In [8]:
!pwd

/Users/anniesu/Documents/school/college_4/SEM_1/CIS_519/cis519_final_project


In [9]:
cd gpt-2

/Users/anniesu/Documents/school/college_4/SEM_1/CIS_519/cis519_final_project/gpt-2


install requirements

In [14]:
!pip3 install -r requirements.txt

download the model

In [11]:
# download the pretrained model
!python download_model.py 345M

Fetching checkpoint: 1.00kit [00:00, 173kit/s]                                                      
Fetching encoder.json: 1.04Mit [00:00, 9.05Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 203kit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [01:29, 15.9Mit/s]                                 
Fetching model.ckpt.index: 11.0kit [00:00, 3.83Mit/s]                                               
Fetching model.ckpt.meta: 927kit [00:00, 7.81Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 4.73Mit/s]                                                       


In [12]:
!export PYTHONIOENCODING=UTF-8

In [16]:
import sys
sys.path.append("/content/gpt-2/src")

In [6]:
import fire
import json
import os
import numpy as np
import tensorflow as tf

import model, sample, encoder

ModuleNotFoundError: No module named 'fire'

In [0]:
import generate_unconditional_samples

In [0]:
import interactive_conditional_samples

In [0]:
class GPT2:

  
  # extracted from the source code to generate some text based on a prior
  def __init__(
      self,
      model_name='345M',
      seed=None,
      nsamples=1,
      batch_size=1,
      length=None,
      temperature=1,
      top_k=0,
      raw_text="",
  ):
      """
      Interactively run the model
      :model_name=117M : String, which model to use
      :seed=None : Integer seed for random number generators, fix seed to reproduce
       results
      :nsamples=1 : Number of samples to return total
      :batch_size=1 : Number of batches (only affects speed/memory).  Must divide nsamples.
      :length=None : Number of tokens in generated text, if None (default), is
       determined by model hyperparameters
      :temperature=1 : Float value controlling randomness in boltzmann
       distribution. Lower temperature results in less random completions. As the
       temperature approaches zero, the model will become deterministic and
       repetitive. Higher temperature results in more random completions.
      :top_k=0 : Integer value controlling diversity. 1 means only 1 word is
       considered for each step (token), resulting in deterministic completions,
       while 40 means 40 words are considered at each step. 0 (default) is a
       special setting meaning no restrictions. 40 generally is a good value.
      """
      if batch_size is None:
          batch_size = 1
      assert nsamples % batch_size == 0

      self.nsamples = nsamples
      self.batch_size = batch_size
      
      self.enc = encoder.get_encoder(model_name)
      hparams = model.default_hparams()
      with open(os.path.join('models', model_name, 'hparams.json')) as f:
          hparams.override_from_dict(json.load(f))

      if length is None:
          length = hparams.n_ctx // 2
      elif length > hparams.n_ctx:
          raise ValueError("Can't get samples longer than window size: %s" % hparams.n_ctx)

      self.sess = tf.Session(graph=tf.Graph())
      self.sess.__enter__()
      
      self.context = tf.placeholder(tf.int32, [batch_size, None])
      np.random.seed(seed)
      tf.set_random_seed(seed)
      self.output = sample.sample_sequence(
          hparams=hparams, length=length,
          context=self.context,
          batch_size=batch_size,
          temperature=temperature, top_k=top_k
      )

      saver = tf.train.Saver()
      self.ckpt = tf.train.latest_checkpoint(os.path.join('models', model_name))
      saver.restore(self.sess, self.ckpt)

  def close(self):
    self.sess.close()
  
  def generate_conditional(self,raw_text):
      context_tokens = self.enc.encode(raw_text)
      generated = 0
      for _ in range(self.nsamples // self.batch_size):
          out = self.sess.run(self.output, feed_dict={
              self.context: [context_tokens for _ in range(self.batch_size)]
          })[:, len(context_tokens):]
          for i in range(self.batch_size):
              generated += 1
              text = self.enc.decode(out[i])
              return text
              #print("=" * 40 + " SAMPLE " + str(generated) + " " + "=" * 40)
              #print(text)
      #print("=" * 80)


In [17]:
gpt2 = GPT2()


INFO:tensorflow:Restoring parameters from models/345M/model.ckpt


In [18]:
result = gpt2.generate_conditional(raw_text="Can you tell me something about music?")

print(result)



SW: Well, I'm an rapping comedian now and the thing I like most about every guy who follows me on Twitter is that whenever I'm stuck in the moment, I'm out there freestyling or tinkering with whatever freakin' notepaper they sold at Twiztid. My favorite example: Two years ago, I tweeted "You people make my fans look like Commie dogs!" I had actually never seen that tweet, and had a hard time telling the difference between the two platforms. My Twitter handle was @itwangoCause… but through those two tweets (which I posted over the course of a few months), I was able to stream over a thousand Twitter followers while simultaneously spreading pure KOREA Weibo envy fablities across the nation.

Since then, I've become an expert on KOREA themes, and have been engaged in a frenzy of blasting myself here and there. All of it made for a thoroughly schmaltzy stream of hilarity, even more efficient than poking fun at myself on Twitter. Occasionally though, it's hard to time your digital attacks

In [0]:
class Who:
  """A class defining the conversation parties: me, he"""
  def __init__(self):
    self.prefixes = []

  def matches(self,phrase):
    for prefix in self.prefixes:
      if phrase.startswith(prefix):
        #print(f"{phrase} starts with {prefix}")
        return True
      
    #print(f"{phrase} does not start with {self.prefixes}")
    return False

  def get_random_prefix(self):
    return self.prefixes[0]
  
class Me(Who):
  def __init__(self):
    super().__init__()
    self.prefixes = ["I said: \""]
   
  
class You(Who):
  def __init__(self):
    super().__init__()
    self.prefixes = ["You said: \""]

In [0]:
class Conversation:
  
  def __init__(self, prior = None):
    if prior is None:
      prior="""
      You said: "Nice to meet you. What's your name?"
      I said: "My name is Pete."
      You said: "That's an interesting name. How old are you?"
      I said: "I'm 40 years old."
      You said: "Can you tell me something about yourself?"
      I said: "Ofcourse! I like playing video games and eating cake. "
      You said: "I like sweet stuff too. What are your plans for tomorrow?"
      """
    self.suggestion = None
    
    self.me = Me()
    self.you = You()
    self.parties  = [ self.me, self.you ]
    
    self.conversation = []
    
    lines = prior.split("\n")
    for line in lines:
      line = line.strip()
      if len(line)!=0:
        party = None
        for party in self.parties:
          if party.matches(line):
            break
        if party is None:
          raise Exception(f"Unknown party: {line}")
                
        self.conversation.append((party,line))
    self.get_suggestion()
    
  
  def get_prior(self):
    conv = ""
    for (party, line) in self.conversation:
      conv+=line+"\n"
    return conv
  
  def get_suggestion(self):
    who, last_line = self.conversation[-1]

    party_index = self.parties.index(who)
    next_party = self.parties[(party_index+1) % len(self.parties)]
      
    conv = self.get_prior()
    conv += next_party.get_random_prefix()
    answer = self.get_answer(next_party, conv)

    if not next_party.matches(answer):
      prefix = next_party.get_random_prefix()
      answer = prefix + answer
    
    self.suggestion = (next_party, answer)
  
  def next(self, party = None, answer = ""):
    """Continue the conversation
    :param party: None -> use the current party which is currently in turn
    :param answer: None -> use the suggestion, specify a text to override the 
           suggestion
    
    """
    suggested_party, suggested_answer = self.suggestion
    if party is None:
      party = suggested_party
    
    if answer == "":
      answer = suggested_answer
      
    if not party.matches(answer):
      prefix = party.get_random_prefix()
      answer = prefix + answer
    
    answer = answer.strip()
    if answer[-1] != "\"":
      # add the closing "
      answer += "\""
      
    self.conversation.append((party, answer))    
    self.get_suggestion()
    
  def retry(self):
    self.get_suggestion()
        
  def get_answer(self, party, conv):
    answer = gpt2.generate_conditional(raw_text=conv)
    lines = answer.split("\n")
    line = ""
    for line in lines:
      if line !="":
        break
      
    if line!="":
      return line
    
    return ""
      
  def show(self):
    conv = ""
    for (party, line) in self.conversation:
      conv+=line+"\n"
    print(conv)
    if self.suggestion is not None:
      party, answer  = self.suggestion
      print("--> "+answer)
    
    


In [0]:
# 'You' is the bot
prior_bot_annie = """
I said: hello. my name is annie. what's your name?
You said: hi. my name is bottie. 
I said: so nice to meet you!
You said: you too!
I said: how old are you?
You said: i'm 20 years old.
I said: nice! i'm 21 years old. i am in college right now.
You said: what's your favorite class?
"""

# Alex is the bot
prior_alex = """
I said: hi Annie. how are you? 
Annie said: Alex, i'm so sorry i could not come get lunch with you and lauren today. hope you had a good thanksgiving break though. also, are you back for christmas?
"""

c = Conversation(prior_alex)

In [32]:
# show the conversation and the suggestion by the ai
c.show()

Annie said: hello. my name is annie. what's your name?
Bottie said: hi. my name is bottie.
Annie said: so nice to meet you!
Bottie said: you too!
Annie said: how old are you?
Bottie said: i'm 20 years old.
Annie said: nice! i'm 21 years old. i am in college right now.
Bottie said: what's your favorite class?

--> I said: "Act I." I said that when i was 19 years old. But i never asked because i didn't want to hurt my voice, and i have carried my ass in rehearsals since then. I love to act on stage.


In [0]:
# "I said" -> answer by the AI
# if the answer of the AI is garbage then call c.retry() 
c.retry()

In [4]:
import random
print(random.random())

0.5519441483542119


In [34]:
c.show()

Annie said: hello. my name is annie. what's your name?
Bottie said: hi. my name is bottie.
Annie said: so nice to meet you!
Bottie said: you too!
Annie said: how old are you?
Bottie said: i'm 20 years old.
Annie said: nice! i'm 21 years old. i am in college right now.
Bottie said: what's your favorite class?

--> I said: "biology"


In [0]:
# Accept the suggested answer by the AI
c.next()

In [0]:

# now its your turn
c.next(c.you, "My favorite pizza is the calzone with lots of cheese.")


In [24]:
c.show()

You said: "Nice to meet you. What's your name?"
I said: "My name is Pete."
You said: "That's an interesting name. How old are you?"
I said: "I'm 40 years old."
You said: "Can you tell me something about yourself?"
I said: "Ofcourse! I like playing video games and eating cake. "
You said: "I like sweet stuff too. What are your plans for tomorrow?"
I said: "Nothing, and I don't know what to do with my life! "

--> You said: "Do you like following the advice of experts? Or do you have all your own ideas?"


In [0]:
c.next()

In [0]:
# now its your turn
c.next(c.you, "You eat pizza often?")

In [0]:
# show the conversation and the reply of the ai
c.show()

You said: "Nice to meet you. What's your name?"
I said: "My name is Pete."
You said: "That's an interesting name. How old are you?"
I said: "I'm 40 years old."
You said: "Can you tell me something about yourself?"
I said: "Ofcourse! I like playing video games and eating cake. "
You said: "I like sweet stuff too. What are your plans for tomorrow?"
I said: "I will participate in playing video games and try to eaten pizza, I'm going to want some apples 'cause I kiss big apples...ill be eating them."
You said: "My favorite pizza is the calzone with lots of cheese."
I said: "Sounds good."
You said: "You eat pizza often?"

--> I said: "Yes, I do. I would like to eat more often...and rock my rocking armor."


In [0]:
# accept ai answer
c.next()

In [0]:
# our reply
c.next(c.you, "Pizza is not to good for your health though.")

In [0]:
# show the conv. and reply of the ai
c.show()

You said: "Nice to meet you. What's your name?"
I said: "My name is Pete."
You said: "That's an interesting name. How old are you?"
I said: "I'm 40 years old."
You said: "Can you tell me something about yourself?"
I said: "Ofcourse! I like playing video games and eating cake. "
You said: "I like sweet stuff too. What are your plans for tomorrow?"
I said: "I will participate in playing video games and try to eaten pizza, I'm going to want some apples 'cause I kiss big apples...ill be eating them."
You said: "My favorite pizza is the calzone with lots of cheese."
I said: "Sounds good."
You said: "You eat pizza often?"
I said: "Yes, I do. I would like to eat more often...and rock my rocking armor."
You said: "Pizza is not to good for your health though."

--> I said: "Oh yeah. I do get so tired from it."


In [0]:
# end the session
gpt2.close()